In [6]:
import pandas as pd
import numpy as np
import random

In [ ]:
cutoffs = pd.read_csv('clgBranchCutoff.csv')
cutoffs = cutoffs.iloc[::2]
cutoffs = cutoffs.drop(columns = ['Unnamed: 0', 'GNT2S', 'LNT2S', 'ORPHAN', 'EWS', 'TFWS', 'ABCD', 'EFGH', 'Unnamed: 16', 'Unnamed: 17'])
cutoffs = cutoffs.iloc[:681]

In [ ]:
clgBranch = pd.read_csv('clgNameBranch.csv')
clg = clgBranch.iloc[::2]
branch = clgBranch.iloc[1::2]
clg = list(clg['clg'])
branch = list(branch['clg'])
cutoffs['ClgName'] = clg
cutoffs['Branch'] = branch
colnames = ['ClgName', 'Branch', 'GOPENS', 'GSCS', 'GSTS', 'GNT1S', 'GOBCS', 'LOPENS', 'LOBCS', 'PWDOPENS']
cutoffs = cutoffs.reindex(columns=colnames)
cutoffs
colname = ['GOPENS', 'GSCS', 'GSTS', 'GNT1S', 'GOBCS', 'LOPENS', 'LOBCS', 'PWDOPENS']
cutoffs = cutoffs.replace(r'\r.*', '', regex=True)
cutoffs = cutoffs.replace(r'.*- ', '', regex=True)
cutoffs

In [ ]:
cutoffs.to_csv("cutoffs.csv")

In [2]:
def extract_score_rank(string):
    info = list(string.split())
    info[1] = info[1][1:-1]
    return float(info[0]),float(info[1])

def extract_name_town_city(sample):
    res = sample.split(',')
    name, town, city = '', '', ''
    if len(res) >= 1: name = res[0]
    if len(res) >= 2: town = res[1]
    if len(res) >= 3: city = res[2]
    return name, town, city


def extraction(file_names: list, extenstions: list):
    res = []
    for index, file_name in enumerate(file_names):
        table = pd.read_csv(file_name)

        for i in range(table.shape[0]):
            if str(table['Score'][i]) == "nan":
                table['ClgName'][i-1] = str(table['ClgName'][i-1])
                table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
        table = table.replace(r'.*- ', '', regex=True)
        table = table.replace(r'.*CET', 'MHTCET', regex=True)
        table = table.replace(r'\n', '', regex=True)


        table = table.drop(columns=['ChoiceCode'])
        table = table.dropna(subset=['Score'])
        
        rank = 'Rank' + extensions[index]
        scores = 'Score' + extensions[index]
        table[rank], table[scores] = zip(*table['Score'].map(extract_score_rank))
        table['Name'], table['Town'], table['City'] = zip(*table['ClgName'].map(extract_name_town_city))
        
        table = table.replace(r'.*- ', '', regex=True)
        
        table = table.drop(columns=['Score', 'ClgName'])
        table = table.reindex(columns=[rank, scores, 'Name', 'Branch', 'Town', 'City', 'Exam'])
        res.append(table)
    return res

In [ ]:
file_names = ['jeecap1.csv', 'jeecap2.csv', 'jeecap1_2021.csv', 'jeecap2_2021.csv']
df = extraction(file_names)

In [24]:
df = pd.read_csv('final.csv')
moreinfo = pd.read_csv('moreclgdata.csv')
moreinfo.drop(moreinfo.iloc[:, 15:],inplace=True, axis=1)
df = pd.merge(df, moreinfo, left_on='Name', right_on='Name')
df = df.drop(columns=['Courses', 'City_x'])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 177
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 178 non-null    int64  
 1   Name                       178 non-null    object 
 2   Branch                     178 non-null    object 
 3   Town                       178 non-null    object 
 4   Exam                       178 non-null    object 
 5   Rank_19_R1                 178 non-null    float64
 6   Score_19_R1                178 non-null    float64
 7   Rank_19_R2                 178 non-null    float64
 8   Score_19_R2                178 non-null    float64
 9   Rank_20_R1                 178 non-null    float64
 10  Score_20_R1                178 non-null    float64
 11  Rank_20_R2                 178 non-null    float64
 12  Score_20_R2                178 non-null    float64
 13  Genders Accepted           178 non-null    object 

In [26]:
df['Rating'] = df['Rating'].astype(float)
df['Rating'][:5]

0    3.77
1    3.77
2     NaN
3     NaN
4     NaN
Name: Rating, dtype: float64

In [32]:
# def assign_ratings(sample, index):
#     if sample: return sample
    
#     rank = df['Rank_20_R1'][index]
#     if rank < 400: df['Rating'] = random.uniform(3.5, 4)
#     elif rank < 800: df['Rating'] = random.uniform(3, 3.5)
#     elif rank < 1200: df['Rating'] = random.uniform(2.5, 3)
#     elif rank < 5000: df['Rating'] = random.uniform(2, 2.5)
#     else: df['Rating'] = random.uniform(1, 2)
    
for i in range(df.shape[0]):
    rank = df['Rank_20_R1'][i]
    if rank < 700: df['Rating'][i] = random.uniform(3.5, 4)
    elif rank < 1100: df['Rating'][i] = random.uniform(3, 3.5)
    elif rank < 2000: df['Rating'][i] = random.uniform(2.5, 3)
    elif rank < 5000: df['Rating'][i] = random.uniform(2, 2.5)
    else: df['Rating'][i] = random.uniform(1, 2)
    
df = df.round({'Rating':2})

<ipython-input-32-a845205bb510>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if rank < 700: df['Rating'][i] = random.uniform(3.5, 4)
<ipython-input-32-a845205bb510>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif rank < 5000: df['Rating'][i] = random.uniform(2, 2.5)
<ipython-input-32-a845205bb510>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif rank < 1100: df['Rating'][i] = random.uniform(3, 3.5)
<ipython-input-32-a

In [33]:
df.head()

,Unnamed: 0,Name,Branch,Town,Exam,Rank_19_R1,Score_19_R1,Rank_19_R2,Score_19_R2,Rank_20_R1,...,Total Faculty,Established Year,Rating,University,Facilities,City_y,State,Country,College Type,Average Fees
0,0,Institute of Chemical Technology,Chemical Engineering,Matunga,JEE,256.0,192.0,391.0,179.0,637.0,...,116,2008,3.55,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205
1,37,Institute of Chemical Technology,Dyestuff Technology,Matunga,JEE,1673.0,128.0,1598.0,130.0,2947.0,...,116,2008,2.28,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205
2,34,Thadomal Shahani Engineering College,Computer Engineering,Bandra,JEE,1025.0,147.0,1159.0,142.0,1073.0,...,110,1983,3.36,"University of Mumbai, Mumbai","Library, Sports, Cafeteria, Auditorium, Medica...",Mumbai,Maharashtra,India,Private,587000
3,38,Thadomal Shahani Engineering College,Information Technology,Bandra,JEE,1882.0,124.0,2076.0,120.0,1446.0,...,110,1983,2.99,"University of Mumbai, Mumbai","Library, Sports, Cafeteria, Auditorium, Medica...",Mumbai,Maharashtra,India,Private,587000
4,74,Thadomal Shahani Engineering College,Chemical Engineering,Bandra,JEE,2496.0,113.0,3169.0,105.0,4015.0,...,110,1983,2.23,"University of Mumbai, Mumbai","Library, Sports, Cafeteria, Auditorium, Medica...",Mumbai,Maharashtra,India,Private,587000


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 177
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 178 non-null    int64  
 1   Name                       178 non-null    object 
 2   Branch                     178 non-null    object 
 3   Town                       178 non-null    object 
 4   Exam                       178 non-null    object 
 5   Rank_19_R1                 178 non-null    float64
 6   Score_19_R1                178 non-null    float64
 7   Rank_19_R2                 178 non-null    float64
 8   Score_19_R2                178 non-null    float64
 9   Rank_20_R1                 178 non-null    float64
 10  Score_20_R1                178 non-null    float64
 11  Rank_20_R2                 178 non-null    float64
 12  Score_20_R2                178 non-null    float64
 13  Genders Accepted           178 non-null    object 

In [35]:
df.to_csv('database.csv')

In [ ]:
file_names = ['jee19-20cap1.csv', 'jee19-20cap2.csv', 'jee20-21cap1.csv', 'jee20-21cap2.csv']
extensions = ['_19_R1', '_19_R2', '_20_R1', '_20_R2']
df = extraction(file_names, extensions)

In [ ]:
df[0].head()

In [ ]:
df[1].head()

In [ ]:
merge_on = ['Name', 'Branch', 'Town', 'City', 'Exam']

res = df[0]
for dataframe in df[1:]:
    res = pd.merge(res, dataframe, left_on=merge_on, right_on=merge_on)

res.head()

In [ ]:
res.columns

In [ ]:
colnames = [ 'Name', 'Branch', 'Town', 'City', 'Exam', 'Rank_19_R1', 'Score_19_R1',
       'Rank_19_R2', 'Score_19_R2', 'Rank_20_R1', 'Score_20_R1', 'Rank_20_R2',
       'Score_20_R2']
res = res.reindex(columns=colnames)

In [ ]:
res.head()

In [ ]:
res.info()

In [ ]:
res.to_csv('final.csv')